In [80]:
# read graphs and build graph pool 
import graph_tool.all as gt

import sys
sys.path.append('/local0/Kariz/code')
import utils.objectstore as object_store

def build_input_format(inputs_str):
    return ' '.join(inputs_str.split(':')).split()

def load_graph_templates(path):
    graph_skeletons = {}
    with open(path, 'r') as fd:
        graph_strs = fd.read().split('#')[1:]
        for g_str in graph_strs:
            g= build_graph_skeleton(g_str)
            graph_skeletons[g.gp.name] = g
    return graph_skeletons


def build_graph_skeleton(g_str):
    g_elements = g_str.split('\n')
    g_name = g_elements[0].split('\t')[1]
    g_id = 0
    g_queuetime = 0

    g = gt.Graph(directed=True)
    g.gp['name'] = g.new_graph_property("string", g_name)
    g.gp['id'] = g.new_graph_property("string", str(g_id))
    g.gp['queue_time'] = g.new_graph_property("int", g_queuetime)
    g.gp['cur_stage'] = g.new_graph_property("int", -1)
    status = g.new_vertex_property("int")
    inputs = g.new_vertex_property("string")
    cache_runtime = g.new_vertex_property("int")
    remote_runtime = g.new_vertex_property("int")
    color = g.new_vertex_property("string")
    ops = g.new_vertex_property("string")
    vids = g.new_vertex_property("int")
    id_v = {}
    
    # build vertices
    for el in g_elements[1:]:
        if el.startswith('v'):
            vid, operation, inputs_str = el.split(',')[1:]
            v = g.add_vertex()
            vids[v] = int(vid)
            id_v[int(vid)] = v
            inputs[v] = inputs_str #build_input_format(inputs_str)
            #print(inputs_str)
            color[v] = '#fb8072' if len(inputs[v]) > 0 else '#80b1d3'
            cache_runtime[v] = 0
            remote_runtime[v] = 0
            ops[v] = operation if operation else 'SAVE'

    # build edges
    for el in g_elements[1:]:
        if el.startswith('e'):
            v_src, v_dest = el.split(',')[1:]
            e = g.add_edge(id_v[int(v_src)], id_v[int(v_dest)])

    g.vp['id'] = vids
    g.vp['color'] = color
    g.vp['tables'] = inputs
    g.vp['remote_runtime'] = remote_runtime
    g.vp['cache_runtime'] = cache_runtime
    g.vp['status'] = status
    g.vp['feature'] = ops
    print('Load query:', g_name)
    return g

graphs_pool = load_graph_templates('/local0/Kariz/expriments/simulator/multidag/%s'%('pig.tpch.template'))

inputs = object_store.load_object_meta('/local0/Kariz/expriments/simulator/multidag/config/inputs.csv')

Load query: TPCH_Q12
Load query: TPCH_Q17
Load query: TPCH_Q9
Load query: TPCH_Q4
Load query: TPCH_Q16
Load query: TPCH_Q8
Load query: TPCH_Q7
Load query: TPCH_Q22
Load query: TPCH_Q19
Load query: TPCH_Q6
Load query: TPCH_Q3
Load query: TPCH_Q5
Load query: TPCH_Q15
Load query: TPCH_Q18
Load query: TPCH_Q13
Load query: TPCH_Q10
Load query: TPCH_Q14
Load query: TPCH_Q20
Load query: TPCH_Q21
Load query: TPCH_Q2
Load query: TPCH_Q11
Load query: TPCH_Q1


In [83]:
import random 

def choose_input(seen_objects):
    reuse = random.choices([1, 0], cum_weights=(reuse_ratio, 1.00), k=1)[0]
    table, _ = random.choice(list(seen_objects.items())) if ((reuse) and (len(seen_objects) > 5)) else random.choice(list(inputs.items())) 
    seen_objects[table] = 1 if table not in seen_objects else seen_objects[table]+1
    return table

def random_query():
    gid, g = random.choice(list(graphs_pool.items()))
    return gid, g.copy()
    
reuse_ratio = 0.32
max_dag_concurrency=10
min_dag_concurrency=1
n_iterations = 10

seen_objects = {}
workload_str=''

for it in range(0, n_iterations):
    n_concurrent_dags = random.randint(min_dag_concurrency, max_dag_concurrency)
    workload_str += ("%" + 'r,%d,%d\n'%(it,n_concurrent_dags))
    for i in range(0, n_concurrent_dags):
        gid, g = random_query()
        workload_str += ('#t,%d%d,%s\n'%(it,i,gid))
        for v in g.vertices():
            if len(g.vp.tables[v]) > 0: 
                table = choose_input(seen_objects) 
                g.vp.tables[v] = table
            workload_str += ('v,%d,%s,%s\n'%(g.vp.id[v],g.vp.feature[v],g.vp.tables[v]))
        for e in g.edges():
            workload_str += ('e,%d,%d\n'%(g.vp.id[e.source()],g.vp.id[e.target()]))
    
print(workload_str)
workload_file = '/local0/Kariz/expriments/simulator/multidag/config/synthetic_worload.g'
with open(workload_file, 'w') as fd:
    fd.write(workload_str)

%r,0,6
#t,00,TPCH_Q7
v,0,MAP_ONLY,f11
v,1,REPLICATED_JOIN:HASH_JOIN,q11
v,2,HASH_JOIN,
v,3,MAP_ONLY,j13
v,4,REPLICATED_JOIN:HASH_JOIN,o15
v,5,GROUP_BY:COMBINER,
v,6,SAMPLER,
v,7,ORDER_BY,
e,0,1
e,1,2
e,2,5
e,3,4
e,4,2
e,5,6
e,6,7
#t,01,TPCH_Q6
v,0,HASH_JOIN,j14
#t,02,TPCH_Q2
v,0,HASH_JOIN,y16
v,1,HASH_JOIN,a3
v,2,HASH_JOIN,m18
v,3,HASH_JOIN,j14
v,4,GROUP_BY,
v,5,SAMPLER,
v,6,ORDER_BY:COMBINER,
v,7,SAVE,
e,0,1
e,1,2
e,2,3
e,3,4
e,4,5
e,5,6
e,6,7
#t,03,TPCH_Q7
v,0,MAP_ONLY,y16
v,1,REPLICATED_JOIN:HASH_JOIN,n19
v,2,HASH_JOIN,
v,3,MAP_ONLY,o15
v,4,REPLICATED_JOIN:HASH_JOIN,p18
v,5,GROUP_BY:COMBINER,
v,6,SAMPLER,
v,7,ORDER_BY,
e,0,1
e,1,2
e,2,5
e,3,4
e,4,2
e,5,6
e,6,7
#t,04,TPCH_Q16
v,0,HASH_JOIN,b3
v,1,HASH_JOIN,m18
v,2,GROUP_BY:COMBINER,
v,3,SAMPLER,
v,4,ORDER_BY,
e,0,1
e,1,2
e,2,3
e,3,4
#t,05,TPCH_Q17
v,0,COGROUP,x16
v,1,GROUP_BY:COMBINER,
e,0,1
%r,1,5
#t,10,TPCH_Q21
v,0,GROUP_BY:MULTI_QUERY,o12
v,1,REPLICATED_JOIN:HASH_JOIN,q17
v,2,MAP_ONLY,n19
v,3,HASH_JOIN,n19
v,4,GROUP_BY:COMBINER,
v